In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
# Read the text file
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [3]:
#Now let’s tokenize the text to create a sequence of words:


tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [4]:
total_words

8200

In [5]:
#Now let’s create input-output pairs by splitting the text into sequences of
#tokens and forming n-grams from the sequences:

input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [6]:
input_sequences

[[1, 1561],
 [1, 1561, 5],
 [1, 1561, 5, 129],
 [1, 1561, 5, 129, 34],
 [647, 4498],
 [647, 4498, 4499],
 [226, 5],
 [226, 5, 1562],
 [6, 827],
 [6, 827, 7],
 [6, 827, 7, 871],
 [1, 234],
 [1, 234, 462],
 [1, 234, 462, 648],
 [6, 110],
 [6, 110, 5],
 [6, 110, 5, 2072],
 [1, 678],
 [1, 678, 1360],
 [1, 678, 1360, 499],
 [1, 294],
 [1, 294, 949],
 [1, 294, 949, 872],
 [1, 56],
 [1, 56, 18],
 [1, 56, 18, 1],
 [1, 56, 18, 1, 1014],
 [1, 56, 18, 1, 1014, 873],
 [1, 577],
 [1, 577, 5],
 [1, 577, 5, 1],
 [1, 577, 5, 1, 423],
 [1, 577, 5, 1, 423, 1361],
 [1, 577],
 [1, 577, 5],
 [1, 577, 5, 1],
 [1, 577, 5, 1, 1785],
 [1, 577, 5, 1, 1785, 763],
 [1, 577],
 [1, 577, 5],
 [1, 577, 5, 1],
 [1, 577, 5, 1, 3185],
 [1, 577, 5, 1, 3185, 649],
 [1, 577],
 [1, 577, 5],
 [1, 577, 5, 1],
 [1, 577, 5, 1, 764],
 [1, 577, 5, 1, 764, 1246],
 [1, 577],
 [1, 577, 5],
 [1, 577, 5, 1],
 [1, 577, 5, 1, 2073],
 [1, 577, 5, 1, 2073, 394],
 [1, 577],
 [1, 577, 5],
 [1, 577, 5, 1],
 [1, 577, 5, 1, 828],
 [1, 577, 5, 

In the above code, the text data is split into lines using the ‘\n’ character as a delimiter. For each line in the text, the ‘texts_to_sequences’ method of the tokenizer is used to convert the line into a sequence of numerical tokens based on the previously created vocabulary. The resulting token list is then iterated over using a for loop. For each iteration, a subsequence, or n-gram, of tokens is extracted, ranging from the beginning of the token list up to the current index ‘i’.

This n-gram sequence represents the input context, with the last token being the target or predicted word. This n-gram sequence is then appended to the ‘input_sequences’ list. This process is repeated for all lines in the text, generating multiple input-output sequences that will be used for training the next word prediction model.

In [7]:
#Now let’s pad the input sequences to have equal length:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))


In the above code, the input sequences are padded to ensure all sequences have the same length. The variable ‘max_sequence_len’ is assigned the maximum length among all the input sequences. The ‘pad_sequences’ function is used to pad or truncate the input sequences to match this maximum length.

The ‘pad_sequences’ function takes the input_sequences list, sets the maximum length to ‘max_sequence_len’, and specifies that the padding should be added at the beginning of each sequence using the ‘padding=pre’ argument. Finally, the input sequences are converted into a numpy array to facilitate further processing.

In [8]:
input_sequences

array([[   0,    0,    0, ...,    0,    1, 1561],
       [   0,    0,    0, ...,    1, 1561,    5],
       [   0,    0,    0, ..., 1561,    5,  129],
       ...,
       [   0,    0,    0, ...,    1, 8198, 8199],
       [   0,    0,    0, ..., 8198, 8199, 3187],
       [   0,    0,    0, ..., 8199, 3187, 3186]], dtype=int32)

In [9]:
#Now let’s split the sequences into input and output:

X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [10]:
X

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1, 1561],
       [   0,    0,    0, ...,    1, 1561,    5],
       ...,
       [   0,    0,    0, ...,   28,    1, 8198],
       [   0,    0,    0, ...,    1, 8198, 8199],
       [   0,    0,    0, ..., 8198, 8199, 3187]], dtype=int32)

In [11]:
y

array([1561,    5,  129, ..., 8199, 3187, 3186], dtype=int32)

In [12]:
#Now let’s convert the output to one-hot encode vectors:

y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [13]:
#Now let’s build a neural network architecture to train the model:

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


The code above defines the model architecture for the next word prediction model. The ‘Sequential’ model is created, which represents a linear stack of layers. The first layer added to the model is the ‘Embedding’ layer, which is responsible for converting the input sequences into dense vectors of fixed size. It takes three arguments:

‘total_words’, which represents the total number of distinct words in the vocabulary;
‘100’, which denotes the dimensionality of the word embeddings;
and ‘input_length’, which specifies the length of the input sequences.
The next layer added is the ‘LSTM’ layer, a type of recurrent neural network (RNN) layer designed for capturing sequential dependencies in the data. It has 150 units, which means it will learn 150 internal representations or memory cells.

Finally, the ‘Dense’ layer is added, which is a fully connected layer that produces the output predictions. It has ‘total_words’ units and uses the ‘softmax’ activation function to convert the predicted scores into probabilities, indicating the likelihood of each word being the next one in the sequence.

In [14]:
#Now let’s compile and train the model:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
3010/3010 [==============================] - 51s 14ms/step - loss: 6.2426 - accuracy: 0.0763
Epoch 2/20
3010/3010 [==============================] - 24s 8ms/step - loss: 5.5156 - accuracy: 0.1238
Epoch 3/20
3010/3010 [==============================] - 24s 8ms/step - loss: 5.1328 - accuracy: 0.1472
Epoch 4/20
3010/3010 [==============================] - 23s 8ms/step - loss: 4.8050 - accuracy: 0.1650
Epoch 5/20
3010/3010 [==============================] - 23s 8ms/step - loss: 4.5066 - accuracy: 0.1819
Epoch 6/20
3010/3010 [==============================] - 23s 8ms/step - loss: 4.2232 - accuracy: 0.2011
Epoch 7/20
3010/3010 [==============================] - 24s 8ms/step - loss: 3.9542 - accuracy: 0.2256
Epoch 8/20
3010/3010 [==============================] - 23s 8ms/step - loss: 3.6960 - accuracy: 0.2553
Epoch 9/20
3010/3010 [==============================] - 24s 8ms/step - loss: 3.4530 - accuracy: 0.2874
Epoch 10/20
3010/3010 [==============================] - 24s 8ms/step - 

In [15]:
seed_text = "I will leave if they"
next_words = 10

In [16]:
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 27ms/step
I will leave if they believed my story to this is the matter however i
